In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Checking the Equivalence of Regular Expressions

In order to check whether two regular expressions $r_1$ and $r_2$ are *equivalent*, perform the 
following steps:
- convert the regular expressions $r_1$ and $r_2$ into non-deterministic *FSMs*
  $F_1$ and $F_2$ such that $L(r_1) = L(F_1)$ and $L(r_2) = L(F_2)$,
- convert the non-deterministic *FSMs* $F_1$ and $F_2$ into deterministic *FSMs*
  $D_1$ and $D_2$ such that $L(D_1) = L(F_1)$ and $L(D_2) = L(F_2)$
- check whether both $L(D_1) \backslash L(D_2)$ and $L(D_2) \backslash L(D_1)$ are empty.

The notebook `Regexp-2-NFA.ipynb` contains the function `RegExp2NFA.toNFA` that can be used to compute a non-deterministic 
<span style="font-variant:small-caps;">Fsm</span> that accepts the language described by a given regular expression.

In [ ]:
import { RegExp2NFA, parseRegex, nfa2dfa, minimizeDFA, DFA, showNFA, dfaToDot } from "./09-Equivalence";

`NFA-2-DFA.ts` contains the function `nfa2dfa` that converts a non-deterministic 
*Fsm* into an equivalent deterministic *Fsm*.

Given two sets `A` and `B`, the function `cartesian_product(A, B)` computes the 
<em style="color:blue">cartesian product</em> $A \times B$ which is defined as
$$ A \times B := \{ (x, y) \mid x \in A \wedge y \in B \}. $$

In [ ]:
function cartesianProduct<S, T>(A: Set<S>, B: Set<T>): Set<[S, T]> {
  const result = new Set<[S, T]>();
  for (const x of A) for (const y of B) result.add([x, y]);
  return result;
}

In [ ]:
const A = new Set([1, 2]);
const B = new Set(["a", "b"]);

const result = cartesianProduct(A, B);
console.log(result);

In [ ]:
type Char = string;
type State = string;
type StatePair = [State, State];

type TransRel1 = Map<string, string>; // key: "q,a" → value: "p"
type TransRel2 = Map<string, string>;

interface DFA1 {
  Q: Set<string>;
  Sigma: Set<Char>;
  delta: TransRel1;
  q0: string;
  F: Set<string>;
}

interface DFA2 {
  Q: Set<string>;
  Sigma: Set<Char>;
  delta: TransRel2;
  q0: string;
  F: Set<string>;
}


Given to deterministic *FSMs* `F1` and `F2`, the expression `fsm_complement(F1, F2)` computes a deterministic 
*FSM* that recognizes the language  $L(F_1)\backslash L(F_2)$.

In [ ]:
function fsmComplement(F1: DFA1, F2: DFA1): DFA2 {
  // ❗ Kein Destructuring — vermeidet tslab-„exports“-Fehler
  const States1 = F1.Q;
  const Sigma = F1.Sigma;
  const delta1 = F1.delta;
  const q1 = F1.q0;
  const A1 = F1.F;

  const States2 = F2.Q;
  const delta2 = F2.delta;
  const q2 = F2.q0;
  const A2 = F2.F;

  // Zustände kombinieren
  const States = cartesianProduct(States1, States2);
  const delta: TransRel2 = new Map();

  for (const [p1, p2] of States) {
    for (const c of Sigma) {
      const next1 = delta1.get(`${p1},${c}`);
      const next2 = delta2.get(`${p2},${c}`);
      if (!next1 || !next2) {
        throw new Error(`Missing transition for (${p1}, ${p2}) on '${c}'`);
      }
      delta.set(`(${p1},${p2}),${c}`, `(${next1},${next2})`);
    }
  }

  // Akzeptierende Zustände: A1 × (States2 \ A2)
  const A2Complement = new Set([...States2].filter((s) => !A2.has(s)));
  const F = new Set(
    [...cartesianProduct(A1, A2Complement)].map(([p1, p2]) => `(${p1},${p2})`)
  );

  const Q = new Set([...States].map(([p1, p2]) => `(${p1},${p2})`));
  const q0 = `(${q1},${q2})`;

  return { Q, Sigma, delta, q0, F };
}

Given a regular expression $r$ and an alphabet $\Sigma$, the function $\texttt{regexp2DFA}(r, \Sigma)$
computes a deterministic *FSM* that accepts
the language specified by $r$.

In [ ]:
type Char = string;
type RegExp =
  | number // 0 für ∅
  | Char
  | [RegExp, ...RegExp[]]; // Tupel-ähnliche Struktur, wie in Python AST

In [ ]:
function regexp2DFA(r: any, Σ: Set<Char>): DFA1 {
  const converter = new RegExp2NFA(Σ);
  const nfa = converter.toNFA(r);
  const dfa = nfa2dfa(nfa);
  return dfa;
}

Given a deterministic *FSM* $F$ the function 
`is_empty(F)` checks whether the language accepted by $F$ is empty.
In this function, the variable `Reachable` is the set of those states that are already known to be reachable
from the start state `q0`. `NewFound` are those states that can be reached from a state in the set 
`Reachable`.  When we find no new states that are reachable, the iteration stops and we check whether
there is a state that is both reachable and acceptable because in that case the language is not empty.

In [ ]:
function isEmpty(F: DFA2): boolean {
  const States = F.Q;
  const Sigma = F.Sigma;
  const delta = F.delta;
  const q0 = F.q0;
  const Accepting = F.F;

  // Menge der bereits erreichbaren Zustände
  const Reachable = new Set<string>([q0]);

  while (true) {
    // Neue erreichbare Zustände finden
    const NewFound = new Set<string>();
    for (const q of Reachable) {
      for (const c of Sigma) {
        const next = delta.get(`${q},${c}`);
        if (next) NewFound.add(next);
      }
    }

    // Falls keine neuen Zustände gefunden → abbrechen
    let allOld = true;
    for (const s of NewFound) {
      if (!Reachable.has(s)) {
        allOld = false;
        break;
      }
    }
    if (allOld) break;

    // Vereinigung beider Mengen
    for (const s of NewFound) Reachable.add(s);
  }

  // Falls einer der akzeptierenden Zustände erreichbar ist → nicht leer
  for (const f of Accepting) {
    if (Reachable.has(f)) return false;
  }

  return true; // keine akzeptierenden Zustände erreichbar
}

The function `regExpEquiv` takes three arguments:
- $r_1$ and $r_2$ are regular expressions,
- $\Sigma$ is the alphabet used in these regular expressions.

The function returns `True` iff $r_1 \doteq r_2$, i.e. if $r_1$ and $r_2$ are equivalent. 

In [ ]:
function regExpEquiv(r1: any, r2: any, Sigma: Set<string>): boolean {
  // Schritt 1: baue DFAs für r1 und r2
  const F1 = regexp2DFA(r1, Sigma);
  const F2 = regexp2DFA(r2, Sigma);

  // Schritt 2: Differenzen der Sprachen bilden
  const r1_minus_r2 = fsmComplement(F1, F2);
  const r2_minus_r1 = fsmComplement(F2, F1);

  // Schritt 3: prüfen, ob beide Differenzen leer sind
  return isEmpty(r1_minus_r2) && isEmpty(r2_minus_r1);
}

The notebook `Test-Equivalence.ipynb` can be used to test this function.